In [4]:
pip install -q rouge-score sentence-transformers pandas google-generativeai tqdm

In [5]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import google.generativeai as genai
from tqdm import tqdm
import json
import time

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv('/content/drive/MyDrive/DS310/eval/eval_no_rewrite.csv')

In [8]:
data

,Question,Answer,Answer_from_chatbot
0,Lộ trình áp dụng mức tiêu chuẩn khí thải đối v...,"Xe ô tô nhập khẩu mới và sản xuất, lắp ráp tiế...","Dựa trên các đoạn thông tin được trích xuất, l..."
1,Trách nhiệm của Bộ Khoa học và Công nghệ trong...,"Bộ Khoa học và Công nghệ Nghiên cứu, rà soát q...",Đoạn trích xuất không chứa thông tin để tôi tr...
2,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho nh...,Nhóm 3: Áp dụng với trường hợp khám sức khỏe đ...,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho cá...
3,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...
4,Những ai đủ điều kiện để đăng ký dự tuyển vào ...,Đối với trình độ cao đẳng: - Người có bằng tốt...,"Để đăng ký dự tuyển vào trình độ cao đẳng, nhữ..."
5,Xe thuần điện (PEV hoặc BEV) được định nghĩa n...,Xe thuần điện (PEV - Pure electric vehicle hoặ...,Xe thuần điện (PEV - Pure electric vehicle hoặ...
6,Xe hybrid điện (HEV) bao gồm những loại cụ thể...,Xe hybrid điện (HEV - Hybrid electric vehicle)...,Xe hybrid điện (HEV - Hybrid electric vehicle)...
7,Thế nào là phương tiện giao thông thông thường?,Phương tiện giao thông thông thường là xe cơ g...,Phương tiện giao thông thông thường là xe cơ g...
8,ấu hiệu nhận biết xe cơ giới sử dụng năng lượn...,Dấu hiệu nhận biết xe cơ giới sử dụng năng lượ...,Để nhận biết xe cơ giới sử dụng năng lượng sạc...
9,Số người tối đa cho phép chở trên mỗi loại xe ...,"Đối với xe ô tô: Số người tối đa cho phép chở,...",Số người tối đa cho phép chở trên mỗi loại xe ...


In [ ]:
EMBED_MODEL = "bkai-foundation-models/vietnamese-bi-encoder"

GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
judge_model = genai.GenerativeModel('gemini-2.5-flash')

In [10]:
class RAGEvaluator:
    def __init__(self):
        self.embedder = SentenceTransformer(EMBED_MODEL)
        self.rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)

    def calculate_scores(self, row):
        ques = str(row['Question'])
        ref = str(row['Answer'])              # Câu trả lời chuẩn (Ground Truth)
        gen = str(row['Answer_from_chatbot']) # Câu trả lời của bot

        rouge_score = self.rouge_scorer.score(ref, gen)['rougeL'].fmeasure

        emb1 = self.embedder.encode(gen, convert_to_tensor=True)
        emb2 = self.embedder.encode(ref, convert_to_tensor=True)
        semantic_score = util.cos_sim(emb1, emb2).item()

        return pd.Series({
            'rouge_l': rouge_score,
            'semantic_score': semantic_score
        })

    def evaluate_dataset(self, df):
        print(f" Đang đánh giá {len(df)} câu hỏi...")
        tqdm.pandas() # Hiện thanh progress bar

        # Áp dụng hàm tính toán cho từng dòng (axis=1)
        results = df.progress_apply(self.calculate_scores, axis=1)

        # Nối kết quả vào bảng gốc
        final_df = pd.concat([df, results], axis=1)
        return final_df

In [11]:
def call_gemini_batch(judge_model, batch_df):
        """Tạo prompt chứa nhiều câu hỏi một lúc"""

        # 1. Tạo chuỗi JSON input
        items = []
        for idx, row in batch_df.iterrows():
            items.append({
                "id": idx, # Giữ ID để dễ tracking
                "question": str(row['Question']),
                "ground_truth": str(row['Answer']),
                "ai_answer": str(row['Answer_from_chatbot'])
            })

        prompt = f"""
        Bạn là giám khảo chấm điểm hệ thống RAG pháp lý.
        Dưới đây là danh sách {len(items)} cặp câu hỏi - trả lời.

        INPUT DATA (JSON):
        {json.dumps(items, ensure_ascii=False)}
        Hãy chấm điểm câu trả lời do chatobt tạo và câu trả lời ground_truth trên thang 1 đến 5, điểm có thể là 3.4, 4.4, không nhất thiết phải là số nguyên dựa trên các tiêu chí:
        1. Tính chính xác về thông tin pháp lý.
        2. Sự đầy đủ so với câu trả lời chuẩn.
        3. Không bịa đặt thông tin.
        4. Hãy so sánh cả về độ đúng trọng tâm, nếu câu trả lời ngắn nhưng đúng trọng tâm thì điểm càng cao
        5. Thứ tự điểm số phải tương ứng chính xác với thứ tự Input.
        6. KHÔNG giải thích, KHÔNG markdown code block, chỉ list số.
        7. Output mẫu: [4.7, 4.3, 3, 5, 1.4]
        """

        try:
            # Gọi API
            response = judge_model.generate_content(prompt)
            text_resp = response.text.strip()

            # Xử lý làm sạch chuỗi nếu model trả về ```json [1,2,3] ```
            text_resp = text_resp.replace("```json", "").replace("```", "").strip()

            # Parse JSON
            scores = json.loads(text_resp)

            # Đảm bảo output là list các số int
            if isinstance(scores, list):
                return scores
            else:
                return [0] * len(items)
        except Exception as e:
            print(f" Lỗi Batch: {e}")
            # Trả về list 0 nếu lỗi
            return [0] * len(items)

In [12]:
# 2. Khởi chạy
evaluator = RAGEvaluator()
df_results = evaluator.evaluate_dataset(data)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

 Đang đánh giá 54 câu hỏi...


100%|██████████| 54/54 [01:14<00:00,  1.37s/it]


In [13]:
df_results

,Question,Answer,Answer_from_chatbot,rouge_l,semantic_score
0,Lộ trình áp dụng mức tiêu chuẩn khí thải đối v...,"Xe ô tô nhập khẩu mới và sản xuất, lắp ráp tiế...","Dựa trên các đoạn thông tin được trích xuất, l...",0.097606,0.804243
1,Trách nhiệm của Bộ Khoa học và Công nghệ trong...,"Bộ Khoa học và Công nghệ Nghiên cứu, rà soát q...",Đoạn trích xuất không chứa thông tin để tôi tr...,0.148936,0.040849
2,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho nh...,Nhóm 3: Áp dụng với trường hợp khám sức khỏe đ...,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho cá...,0.835165,0.823402
3,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,0.637143,0.842173
4,Những ai đủ điều kiện để đăng ký dự tuyển vào ...,Đối với trình độ cao đẳng: - Người có bằng tốt...,"Để đăng ký dự tuyển vào trình độ cao đẳng, nhữ...",0.299233,0.728039
5,Xe thuần điện (PEV hoặc BEV) được định nghĩa n...,Xe thuần điện (PEV - Pure electric vehicle hoặ...,Xe thuần điện (PEV - Pure electric vehicle hoặ...,1.000000,0.991718
6,Xe hybrid điện (HEV) bao gồm những loại cụ thể...,Xe hybrid điện (HEV - Hybrid electric vehicle)...,Xe hybrid điện (HEV - Hybrid electric vehicle)...,0.836090,0.947159
7,Thế nào là phương tiện giao thông thông thường?,Phương tiện giao thông thông thường là xe cơ g...,Phương tiện giao thông thông thường là xe cơ g...,1.000000,1.000000
8,ấu hiệu nhận biết xe cơ giới sử dụng năng lượn...,Dấu hiệu nhận biết xe cơ giới sử dụng năng lượ...,Để nhận biết xe cơ giới sử dụng năng lượng sạc...,0.449799,0.780287
9,Số người tối đa cho phép chở trên mỗi loại xe ...,"Đối với xe ô tô: Số người tối đa cho phép chở,...",Số người tối đa cho phép chở trên mỗi loại xe ...,0.515060,0.834515


In [18]:
BATCH_SIZE = 10
total_rows = len(data)
all_scores = []
print(f"\n Bắt đầu gọi API chấm điểm (Batch Size = {BATCH_SIZE})...")

for i in tqdm(range(0, total_rows, BATCH_SIZE)):
  batch_df = df_results.iloc[i : i + BATCH_SIZE]
  batch_scores = call_gemini_batch(judge_model,batch_df)

  if len(batch_scores) != len(batch_df):
    print(f"Batch {i} gửi {len(batch_df)} dòng nhưng nhận {len(batch_scores)} điểm. Fill 0 cho an toàn.")
    batch_scores = [0] * len(batch_df)

  all_scores.extend(batch_scores)

  # Ngủ để tránh bị Google chặn vì spam request quá nhanh
  time.sleep(60)

df_results['llm_score'] = all_scores



 Bắt đầu gọi API chấm điểm (Batch Size = 10)...


100%|██████████| 6/6 [09:32<00:00, 95.39s/it]


In [19]:
print("\n=== KẾT QUẢ ĐÁNH GIÁ TRUNG BÌNH ===")
print(f"ROUGE-L (Cấu trúc): {df_results['rouge_l'].mean():.4f}")
print(f"Semantic (Ngữ nghĩa): {df_results['semantic_score'].mean():.4f}")
print(f"LLM Score: {df_results['llm_score'].mean():.4f}")



=== KẾT QUẢ ĐÁNH GIÁ TRUNG BÌNH ===
ROUGE-L (Cấu trúc): 0.4332
Semantic (Ngữ nghĩa): 0.6807
LLM Score: 3.3944


In [16]:
df_results

,Question,Answer,Answer_from_chatbot,rouge_l,semantic_score,llm_score
0,Lộ trình áp dụng mức tiêu chuẩn khí thải đối v...,"Xe ô tô nhập khẩu mới và sản xuất, lắp ráp tiế...","Dựa trên các đoạn thông tin được trích xuất, l...",0.097606,0.804243,3.0
1,Trách nhiệm của Bộ Khoa học và Công nghệ trong...,"Bộ Khoa học và Công nghệ Nghiên cứu, rà soát q...",Đoạn trích xuất không chứa thông tin để tôi tr...,0.148936,0.040849,1.0
2,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho nh...,Nhóm 3: Áp dụng với trường hợp khám sức khỏe đ...,Tiêu chuẩn sức khỏe Nhóm 3 được áp dụng cho cá...,0.835165,0.823402,5.0
3,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,Cấu trúc dữ liệu kết quả khám sức khỏe của ngư...,0.637143,0.842173,2.0
4,Những ai đủ điều kiện để đăng ký dự tuyển vào ...,Đối với trình độ cao đẳng: - Người có bằng tốt...,"Để đăng ký dự tuyển vào trình độ cao đẳng, nhữ...",0.299233,0.728039,4.0
5,Xe thuần điện (PEV hoặc BEV) được định nghĩa n...,Xe thuần điện (PEV - Pure electric vehicle hoặ...,Xe thuần điện (PEV - Pure electric vehicle hoặ...,1.000000,0.991718,5.0
6,Xe hybrid điện (HEV) bao gồm những loại cụ thể...,Xe hybrid điện (HEV - Hybrid electric vehicle)...,Xe hybrid điện (HEV - Hybrid electric vehicle)...,0.836090,0.947159,4.8
7,Thế nào là phương tiện giao thông thông thường?,Phương tiện giao thông thông thường là xe cơ g...,Phương tiện giao thông thông thường là xe cơ g...,1.000000,1.000000,5.0
8,ấu hiệu nhận biết xe cơ giới sử dụng năng lượn...,Dấu hiệu nhận biết xe cơ giới sử dụng năng lượ...,Để nhận biết xe cơ giới sử dụng năng lượng sạc...,0.449799,0.780287,3.5
9,Số người tối đa cho phép chở trên mỗi loại xe ...,"Đối với xe ô tô: Số người tối đa cho phép chở,...",Số người tối đa cho phép chở trên mỗi loại xe ...,0.515060,0.834515,4.5


In [17]:
df_results.to_csv("ket_qua_danh_gia.csv", index=False)